In [2]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch


# Check device availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

excel_path = 'Data_all.xlsx'
questions = pd.read_excel('question.xlsx')
answers = pd.read_excel(excel_path)
data = pd.merge(questions, answers, on='id_question')
data.dropna(subset=['stu_answer','grade','question'], inplace=True)

answer = data['stu_answer'].str.replace(r"[\'\"\!\#\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~\t\n\`÷×؛<>_()\*\&\^%\]\[ـ،\/\:\؟\.\,'{}\~¦\+\|!”…“–ـ]", "")
print(answer)

0                                                 رمضان
1                                             شهر رمضان
2                                             شهر الصوم
3                             الشهر الذي نزل فيه القرآن
4                 شهر العبادة والتقرب إلى الله هو رمضان
                             ...                       
1415       طالت دعوة الرسول صلى الله عليه وسلم 23 عاماً
1416      استغرقت دعوة الرسول صلى الله عليه وسلم 23 سنة
1417    استغرقت دعوة الرسول صلى الله عليه وسلم 23 عاماً
1418                                   استغرقت 23 عاماً
1419                                     استغرقت 23 سنة
Name: stu_answer, Length: 1411, dtype: object


C:\Users\AzComputer\AppData\Local\Temp\ipykernel_17792\1986856743.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  answer = data['stu_answer'].str.replace(r"[\'\"\!\#\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~\t\n\`÷×؛<>_()\*\&\^%\]\[ـ،\/\:\؟\.\,'{}\~¦\+\|!”…“–ـ]", "")


In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch


# Check device availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

excel_path = 'Data_all.xlsx'
questions = pd.read_excel('question.xlsx')
answers = pd.read_excel(excel_path)
data = pd.merge(questions, answers, on='id_question')
data.dropna(subset=['stu_answer','grade','question'], inplace=True)

answer = data['stu_answer'].str.replace(r"[\'\"\!\#\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~\t\n\`÷×؛<>_()\*\&\^%\]\[ـ،\/\:\؟\.\,'{}\~¦\+\|!”…“–ـ]", "")
question = data['question'].tolist()
answer = data['stu_answer'].tolist()
grade = data['grade'].tolist()

training_data = [f"Q: {q}\nA: {a}\nGrade: {g}" for q, a, g in zip(question, answer, grade)]
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenized_data = tokenizer(training_data, return_tensors='pt', padding=True, truncation=True)

labels = tokenized_data['input_ids']
print(labels.shape)
dataset = TensorDataset(labels)


train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.vocab_size = model.config.n_embd
model.resize_token_embeddings(len(tokenizer))

# Move model to device
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Number of training epochs
num_epochs = 5

# Loop over epochs
for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    
    # Initialize loss
    total_loss = 0.0
    
    # Loop over batches in the training data loader
    for batch in train_dataloader:
        # Move batch to device
        input_ids = batch[0].to(device)
        input_ids = input_ids[:, :-1].contiguous()
        labels = batch[0].to(device)
        labels = labels[:, 1:].contiguous()
        # Reset gradients
        optimizer.zero_grad()
        # Forward pass
        print(input_ids.shape)
        print(labels.shape)
        outputs = model(input_ids, labels=labels)
        
        # Compute loss
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        # Accumulate loss
        total_loss += loss.item()

    # Compute average training loss
    avg_train_loss = total_loss / len(train_dataloader)

    # Set model to evaluation mode
    model.eval()
    
    # Initialize validation loss
    val_loss = 0.0
    
    # Loop over batches in the validation data loader
    with torch.no_grad():
        for batch in val_dataloader:
            # Move batch to device
            input_ids = batch[0].to(device)
            input_ids = input_ids[:, :-1].contiguous()
            labels = batch[0].to(device)
            labels = labels[:, 1:].contiguous()
            
            # Forward pass
            outputs = model(input_ids, labels=labels)
            
            # Accumulate validation loss
            val_loss += outputs.loss.item()

    # Compute average validation loss
    avg_val_loss = val_loss / len(val_dataloader)

    # Print epoch summary
    print(f'Epoch {epoch+1}/{num_epochs}, Avg. Training Loss: {avg_train_loss:.4f}, Avg. Validation Loss: {avg_val_loss:.4f}')

    # Save the model and tokenizer after each epoch
    model.save_pretrained(f'arabic_qa_gpt_model_epoch_{epoch+1}')
    tokenizer.save_pretrained(f'arabic_qa_gpt_model_epoch_{epoch+1}')

C:\Users\AzComputer\AppData\Local\Temp\ipykernel_17792\4032227798.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  answer = data['stu_answer'].str.replace(r"[\'\"\!\#\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~\t\n\`÷×؛<>_()\*\&\^%\]\[ـ،\/\:\؟\.\,'{}\~¦\+\|!”…“–ـ]", "")


torch.Size([1411, 282])


c:\Users\AzComputer\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8, 281])
torch.Size([8

In [84]:
# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('arabic_qa_gpt_model_epoch_5')  # replace with your model path
tokenizer = GPT2Tokenizer.from_pretrained('arabic_qa_gpt_model_epoch_5')  # replace with your tokenizer path

# Move model to the right device
model = model.to(device)

# Prepare a question and a given answer for testing
question = "ما هو الشهر الذي يصوم فيه المسلمون  "


 # replace with your question
given_answer = "شالشهر  هو  رمضان "
 # replace with the given answer
grade = 2# replace with the given grade
# Condition the input data on the grade


input_data = f"Q: {question}\nA: {given_answer}\n  "
print(f"Input data: {input_data}")
# Encode the input data
input_ids = tokenizer.encode(input_data, return_tensors='pt', padding='max_length', max_length=100, truncation=True)
print(f"Input IDs: {input_ids}")
# Create attention mask
attention_mask = input_ids != tokenizer.pad_token_id

# Move the input and attention mask to the right device
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Generate a corrected answer
output = model.generate(input_ids, attention_mask=attention_mask, max_length=200, temperature=0.5, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, do_sample=True)
print(f"Output: {output}")

# Decode the corrected answer
corrected_answer = tokenizer.decode(output[0], skip_special_tokens=True)


print(corrected_answer)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Input data: Q: ما هو الشهر الذي يصوم فيه المسلمون  
A: شالشهر  هو  رمضان 
  
Input IDs: tensor([[   48,    25, 47048, 12919, 18923,   229, 30335, 28981,   148,   112,
         29519, 26897, 28981,   148,   108, 22654, 18923,   232,   148,   113,
         30335, 25405, 18923,   223, 22654, 29519, 28981, 25405, 45692, 13862,
         25405, 30335, 23338,   220,   220,   198,    32,    25, 17550,   112,
         23525,   148,   112, 29519, 26897,   220, 18923,   229, 30335,   220,
         17550,   109, 25405,   148,   114, 12919, 23338,   220,   198,   220,
           220, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]])
Output: tensor([[   48,    25, 47048, 12919, 18923,   229, 30335, 28981,   148,   112,
         29519, 26897, 28981,   1